In [12]:
import numpy as np
from numpy import linalg as LA
from ipykernel import kernelapp as app
import random

In [ ]:
#### Parte de DFP

In [7]:
def cuadrados(x):
    resultado=0
    for i in range(len(x)):
        resultado+=x[i]**2
    return resultado

In [8]:
def Grad(f, x0, h=1e-6, i=-1):
    """
    Función que calcula el Grad de una función en un punto
    """
    n = len(x0)
    if i in range(n):
        z = np.zeros(n)
        z[i] = h/2
        Grad = (f(x0 + z) - f(x0 - z))/h
    else:
        Grad = np.zeros(n)
        for j in range(n):
            z = np.zeros(n)
            z[j] = h/2
            Grad[j] = (f(x0 + z) - f(x0 - z))/h
    return np.array(Grad)

In [9]:
def Hess(f, x0, h=1e-4, method="basic"):
    """
    Función que calcula la Hessiana  de una función en un punto.
    f: función sobre la cual queremos calcular la hessiana.
    x0: Punto sobre el cual queremos hacer el cálculo
    h: nivel de precisión para hacer el cálculo
    method: Método por el cual se quiere hacer puede ser:
             'basic', 'grad', 'centered', 'gradCentered'
    """
    n = len(x0)
    Hess = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            z_i = np.zeros(n)
            z_i[i] = h
            z_j = np.zeros(n)
            z_j[j] = h
            if method == "basic":
                Hess[i, j] = (f(x0 + z_j + z_i) - f(x0 + z_i) -
                              f(x0+z_j) + f(x0)) / (h**2)
            elif method == "grad":
                Hess[i, j] = (Grad(f, x0+z_j, h, i) - Grad(f, x0, h, i) +
                              Grad(f, x0+z_i, h, j) - Grad(f, x0, h, j))/(2*h)
            elif method == "centered":
                if i == j:
                    Hess[i, j] = (-f(x0+2*z_i) + 16*f(x0+z_i) - 30*f(x0) +
                                  16*f(x0-z_i) - f(x0-2*z_i)) / (12*h**2)
                else:
                    Hess[i, j] = (f(x0+z_i+z_j) - f(x0 + z_i - z_j) -
                                  f(x0 - z_i + z_j) + f(x0-z_i-z_j))/(4*h**2)
            elif method == "gradCentered":
                Hess[i, j] = (Grad(f, x0+z_j, h)[i] - Grad(f, x0-z_j, h)[i] +
                              Grad(f, x0+z_i, h)[j] - Grad(f, x0-z_i, h)[j])\
                               / (4 * h)
    return Hess

In [15]:
def genera_alpha(f, x0, pk, c1=1e-4, tol=1e-5):
    """
    Backtracking LS i.e. Algoritmo que encuentra una alpha que cumpla condiciones de wolfe. 
    """
    alpha, rho, c = 1, 4/5, c1
    while f(x0 + alpha*pk)>f(x0) + c*alpha*np.dot(Grad(f, x0),pk):
        alpha*=rho
    return alpha

In [10]:
# Condicion de DFP en HK
def DFP_Hk(yk, sk, Hk):
    """
    Función que calcula La actualización DFP de la matriz Hk
    In:
      yk: Vector n
      sk: Vector n
      Hk: Matriz nxn
    Out:
      Hk+1: Matriz nxn
    """
    Hk1 = Hk - (Hk * yk * yk.T * Hk)/(yk.T * Hk * yk) + (sk * sk.T)/(yk.T * sk)
    return Hk1

In [16]:
#Algoritmo de BFGS con actualizacion de Hk de DFP
def BFGS_DFP(f, x0, tol, H0, maxiter=100):
    k = 0
    Gk = Grad(f, x0)
    Hk = H0
    xk = np.array(x0)
    xk1 = np.array(x0)
    sk = np.array(100)
    while (LA.norm(Gk) > tol and LA.norm(sk) > tol):
        pk = - Hk.dot(Gk)
        alphak = genera_alpha(f, xk, pk)
        xk1 = xk + alphak * pk
        sk = xk1 - xk
        Gk1 = Grad(f, xk1)
        yk = Gk1 - Gk
        Hk = DFP_Hk(yk, sk, Hk) ### Actualizacion DFP
        k += 1
        xk = xk1
        Gk = Gk1
    return xk1, k

In [13]:
x0=np.array([(-1)**i*10 for i in range(10)])

In [102]:
# Parte 1
x, k = BFGS_DFP(cuadrados, x0, 1e-20, np.eye(10))

C:\Users\Joshi\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in divide


In [103]:
x

array([ 10., -10.,  10., -10.,  10., -10.,  10., -10.,  10., -10.])

In [ ]:
#### Parte 2 GC

In [99]:
random.seed(164181)
n,m=100,1000000 #  n para probar pues se tarda con m
Diag_A=[random.randint(1,1000) for x in range(m)]
b=[random.randint(1,1000) for x in range(m)]

In [51]:
def gradiente_conjugado(x0, A, b):
    xk = x0
    b = np.matrix(b).T
    rk = np.dot(A, x0) - b
    pk = -rk
    while not (rk.T * rk ==  0):
        alphak = rk.T * rk / (pk.T * A * pk)
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -rk_1 + betak_1 * pk
        xk, rk, pk = xk_1, rk_1, pk_1
    return xk


In [52]:
def gradiente_conjugado_precond(x0, A, b, M):
    xk = x0
    b = np.matrix(b).T
    rk = np.dot(A, x0) - b
    yk = np.linalg.solve(M, rk)
    pk = -yk 
    while not (rk.T * rk ==  0):
        alphak = rk.T * yk / (pk.T * A * pk)
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        yk_1 = np.linalg.solve(M, rk_1)
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -yk_1 + betak_1 * pk
        xk, rk, pk, yk  = xk_1, rk_1, pk_1, yk_1
    return xk

In [100]:
x_0=np.zeros(m)

In [101]:
# Se modificaron las dimensiones de las operaciones para estar definidas y obtener un resultado
def gradiente_conjugado_mod(x0, A, b):
    k=0
    xk = np.array(x0)
    A=np.array(A)
    b=np.array(b)
    rk = A*x0 - b
    pk = -rk
    while not (LA.norm(rk)==  0):
        alphak = LA.norm(rk)**2 / (np.dot(A,pk**2))
        alphak= alphak
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        betak_1 = LA.norm(rk_1)**2 / LA.norm(rk)**2
        betak_1 = betak_1
        pk_1 = -rk_1 + betak_1 * pk
        xk, rk, pk,k = xk_1, rk_1, pk_1,k+1
    return xk,k

In [97]:
resultado,iteraciones=gradiente_conjugado_mod(x_0, Diag_A, b)

In [98]:
print(LA.norm((Diag_A*resultado)-b), iteraciones) # converge a 0 la norma en esas interaciones

(2.1341964563565222e-12, 680)
